### Convert the tiff files to NifTi 

In [ ]:
"""Fix the spacings to the minimum among among all the datasets. This alleviates the need for resampling during both training and inference. This is mandotory where 
the images are very large(>2GB in uncpompressed format) and cannot be resampled on the fly due to memory issues."""


import numpy as np
import nibabel as nib
import os
import tifffile as tiff
import SimpleITK as sitk

img_path="/home/cellsmb/Desktop/Dinuka/Image_Analysis/New_IF_Dataset/New_IF_Dataset" #main folder for all images
seg_path="/home/cellsmb/Desktop/Dinuka/Image_Analysis/New_IF_Dataset/New_IF_Dataset" #main folder for segmentation files
raw_path ="/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw" #where to save the processed files

file_indexes = {"Endo.tif":0,"Myo_Registered.tif":1,"Nuclei_Registered.tif":2, }
IMAGE_ID = 0


if not os.path.exists(raw_path):
    os.makedirs(raw_path)
    os.makedirs(os.path.join(raw_path,'labelsTr'))
    os.makedirs(os.path.join(raw_path,'imagesTr'))

min_spacing = (0,0,0)
max_spacing =(0,0,0)

num_itterations = 0
for folder in os.listdir(seg_path):
    if os.path.isdir(os.path.join(seg_path, folder)):
        for files in os.listdir(os.path.join(seg_path, folder)):
            if files.endswith(".nii"):
                file = sitk.ReadImage(os.path.join(seg_path, folder, files))
                spacing = file.GetSpacing()
                if num_itterations == 0:
                    min_spacing = spacing
                    max_spacing = spacing
                else:
                    min_spacing = np.minimum(min_spacing, spacing)
                    max_spacing = np.maximum(max_spacing, spacing)
                num_itterations += 1
print("Min Spacing: ", min_spacing)
print("Max Spacing: ", max_spacing)

for folder in os.listdir(seg_path):
    if os.path.isdir(os.path.join(seg_path, folder)):
        IMAGE_ID += 1
        print("Processing folder: ", folder, " with IMAGE_ID: ", IMAGE_ID)
        for files in os.listdir(os.path.join(seg_path, folder)):
            if files.endswith(".nii"):
                seg_file_path=os.path.join(seg_path, folder, files)
                ref_file=sitk.ReadImage(seg_file_path)
                ref_file_again = nib.load(seg_file_path)

                ref_file_again.header["dim"][4:]=1
                ref_file_again.header["xyzt_units"]=2
                ref_file_again.header["sform_code"]=1

                # if np.any([(a/b)>1 for a,b in zip(ref_file.GetSpacing(), min_spacing)]):
                new_spacing = min_spacing
                new_affine = np.diag([min_spacing[0], min_spacing[1], min_spacing[2], 1.0])
                # else:
                #     new_spacing = ref_file.GetSpacing()
                #     new_affine = ref_file_again.affine
                ref_file_again = nib.Nifti1Image(ref_file_again.get_fdata().astype(np.uint8), new_affine, ref_file_again.header)
                ref_file_again.header.set_zooms(new_spacing)

                nib.save(ref_file_again, os.path.join(raw_path,"labelsTr", "IF_Tr_%0.4d.nii.gz"%IMAGE_ID))
                break

        for files in os.listdir(os.path.join(img_path, folder)):
            if os.path.isdir(os.path.join(img_path, folder, files)):
                for file in os.listdir(os.path.join(img_path, folder, files)):
                    if file.endswith(".tif"):
                        for key in file_indexes.keys():
                            if file.endswith(key):
                                file_ID = file_indexes[key]
                                tif_file=os.path.join(img_path, folder, files, file)
                                tif_file=tiff.imread(tif_file).astype(np.uint8)

                                sitk_image = sitk.GetImageFromArray(tif_file)
                                sitk_image.SetSpacing(new_spacing) #ref_file.GetSpacing())
                                sitk_image.SetOrigin(ref_file.GetOrigin()) #ref_file.GetOrigin())
                                sitk_image.SetDirection(ref_file.GetDirection()) #ref_file.GetDirection())
                                tmp_path = os.path.join(raw_path,"imagesTr", "IF_Tr_%0.4d_%0.4d.nii.gz"%(IMAGE_ID,file_ID))
                                sitk.WriteImage(sitk_image, tmp_path)
                                del sitk_image
                                del tif_file

                                img_again = nib.load(tmp_path)
                                img_again = nib.Nifti1Image(img_again.get_fdata(), new_affine,img_again.header)
                                nib.save(img_again, tmp_path)

            else:
                if files.endswith(".tif"):
                    tif_file=os.path.join(img_path, folder, files)
                    tif_file=tiff.imread(tif_file).astype(np.uint8)

                    sitk_image = sitk.GetImageFromArray(tif_file)
                    sitk_image.SetSpacing(new_spacing)
                    sitk_image.SetOrigin(ref_file.GetOrigin()) #ref_file.GetOrigin())
                    sitk_image.SetDirection(ref_file.GetDirection()) #ref_file.GetDirection())
                    sitk.WriteImage(sitk_image, os.path.join(raw_path,"imagesTr", "IF_Tr_%0.4d_0000.nii.gz"%IMAGE_ID))

Processing folder:  YJ_WIF#7 TTNKI#186 15ps WT  with IMAGE_ID:  1
Processing folder:  E9.5  with IMAGE_ID:  2
Processing folder:  YJ_WIF#16 TTNKI 588 8ps WT  with IMAGE_ID:  3
Processing folder:  YJ_WIF#16 TTNKI 597 13ps WT  with IMAGE_ID:  4
Processing folder:  YJ_WIF#16 TTNKI 599 10ps WT  with IMAGE_ID:  5
Processing folder:  YJ_WIF#16 TTNKI;mT#353 19ps WT  with IMAGE_ID:  6


In [19]:
import nibabel as nib
import os
import numpy as np
folder = "/home/cellsmb/Desktop/Dinuka/Image_Analysis/Model_results/Model_results/IF_352_6img_results/all_segs_no_bg"
new_affine = np.diag([min_spacing[0], min_spacing[1], min_spacing[2], 1.0])

for file in os.listdir(folder):
    if file.endswith(".nii.gz"):
        img = nib.load(os.path.join(folder, file))
        img = nib.Nifti1Image(img.get_fdata().astype(np.uint8), new_affine, img.header)
        nib.save(img, os.path.join(folder, file))  # Overwrite the original file

In [1]:
#change the file name ; Increment the IMAGE_ID by 1

import os
import SimpleITK as sitk
import nibabel as nib
from nibabel.orientations import aff2axcodes, io_orientation, axcodes2ornt, ornt_transform
path = "/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw/imagesTr/IF_Tr_0001_0000.nii.gz"
seg = "/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw/labelsTr/IF_Tr_0003.nii.gz"
img = nib.load(path)
seg=nib.load(seg)

print(seg.affine)
print(img.affine)
print(aff2axcodes(seg.affine))
print(aff2axcodes(img.affine))

[[0.27000001 0.         0.         0.        ]
 [0.         0.27000001 0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]]
[[0.27000001 0.         0.         0.        ]
 [0.         0.27000001 0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]]
('R', 'A', 'S')
('R', 'A', 'S')


In [2]:
import numpy as np
np.unique(seg.get_fdata())

array([1., 2., 3., 4., 5.])

In [ ]:
#### CONVERT THE SEGMENTATION LABELS

import numpy as np
import nibabel as nib
import os

path="/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw/labelsTr"

for files in os.listdir(path):
    if files.endswith(".nii.gz"):
        seg_file_path=os.path.join(path, files)
        print(seg_file_path)
        seg=nib.load(seg_file_path)
        seg_file=seg.get_fdata().astype(np.uint8)
        
        seg_file[seg_file==1]=0
        seg_file[seg_file==2]=1
        seg_file[seg_file==3]=2
        seg_file[seg_file==4]=3
        seg_file[seg_file==5]=4
        seg_file=nib.Nifti1Image(seg_file.astype(np.uint8), seg.affine, seg.header)
        nib.save(seg_file, os.path.join(path,files))


/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw/labelsTr/IF_Tr_0002.nii.gz


In [1]:
import numpy as np
import nibabel as nib
import os

img = nib.load("/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw/labelsTr/IF_Tr_0002.nii.gz")
arr = img.get_fdata().astype(np.uint8)
arr[arr==4]=7
arr[arr==5]=4
arr[arr==7]=5
print("Dome")
arr = nib.Nifti1Image(arr.astype(np.uint8), img.affine, img.header)
nib.save(arr, "/home/cellsmb/Desktop/Dinuka/Image_Analysis/nnUnet_raw/labelsTr/IF_Tr_0002.nii.gz")

Dome


In [ ]:
import nibabel as nib
import numpy as np

seg= nib.load("/Users/dbattagodage/Desktop/Datasets/nnUnet_raw/Dataset352_IF/labelsTr/IF_Tr_0003.nii.gz").get_fdata().astype(np.uint8)
np.unique(seg)

In [ ]:
## Change the dtype

import numpy as np
import nibabel as nib
import os
new_path="/Users/dbattagodage/Desktop/Datasets/nnUNet_raw/Dataset355_Endocardium"
path="/Users/dbattagodage/Desktop/Datasets/nnUNet_raw/Dataset345_Endocardium"
os.mkdir(os.path.join(path, "imagesTr_2"))

for files in os.listdir(os.path.join(path, "imagesTr")):
    if files.endswith(".nii.gz"):
        file_path=os.path.join(path, "imagesTr", files)
        nib_file=nib.load(file_path)
        nib_file_data=nib_file.get_fdata().astype(np.float32)
        nib_image=nib.Nifti1Image(nib_file_data, nib_file.affine, nib_file.header)
        nib_image.set_data_dtype(np.float32)
        nib.save(nib_image, os.path.join(path, "imagesTr_2", files))